In [1]:
#일자별
import  warnings
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys 
from    selenium.webdriver.common.action_chains import ActionChains
from    selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from    bs4      import BeautifulSoup
import  time
import  pyperclip
import  requests
import  datetime
import  pymssql
import  pandas as pd
import  chromedriver_autoinstaller
import  subprocess
from    selenium import webdriver
from    selenium.webdriver.chrome.options import Options
from    selenium.webdriver.common.alert import Alert
import  chromedriver_autoinstaller
import  subprocess
import  shutil 
import  xlrd
import  openpyxl

warnings.filterwarnings('ignore')

conn = pymssql.connect(server='localhost', user='pyuser', password='Dnflskfk12!', database='mytest' , charset='utf8') 

itembay_ragnarok_query = '''

IF OBJECT_ID(N'tempdb..#itembay_ragnarok') IS NOT NULL
DROP TABLE #itembay_ragnarok 

CREATE TABLE #itembay_ragnarok (
game	nvarchar(max) null,
server	nvarchar(max) null,
is_trade_finished nvarchar(max) null,
zeny_sell_title	 nvarchar(max) null,
zeny nvarchar(max) null,
cash bigint null,
regdate nvarchar(max) null
)

bulk insert #itembay_ragnarok  from 'C:/Users/Gravity/Desktop/python/raw/itembay_ragnarok.csv'  with (firstrow=2, fieldterminator = ',' , rowterminator = '\n', codepage='65001' , keepnulls )

insert into mytest.dbo.zeny_selling_monitoring
select * FROM #itembay_ragnarok

select * FROM mytest.dbo.zeny_selling_monitoring_2 with (nolock) ;
'''

itembay_ragnarok_zero_query = '''

IF OBJECT_ID(N'tempdb..#itembay_ragnarok_zero') IS NOT NULL
DROP TABLE #itembay_ragnarok_zero 

CREATE TABLE #itembay_ragnarok_zero (
game	nvarchar(max) null,
server	nvarchar(max) null,
is_trade_finished nvarchar(max) null,
zeny_sell_title	 nvarchar(max) null,
zeny nvarchar(max) null,
cash bigint null,
regdate nvarchar(max) null
)

bulk insert #itembay_ragnarok_zero  from 'C:/Users/Gravity/Desktop/python/raw/itembay_ragnarok_zero.csv'  with (firstrow=2, fieldterminator = ',' , rowterminator = '\n', codepage='65001' , keepnulls )

insert into mytest.dbo.zeny_selling_monitoring
select * FROM #itembay_ragnarok_zero 

select * FROM mytest.dbo.zeny_selling_monitoring_2 with (nolock) ; 
'''

itemmania_ragnarok_query = '''

IF OBJECT_ID(N'tempdb..#itemmania_ragnarok') IS NOT NULL
DROP TABLE #itemmania_ragnarok 

CREATE TABLE #itemmania_ragnarok (
game	nvarchar(max) null,
server	nvarchar(max) null,
is_trade_finished nvarchar(max) null,
zeny_sell_title	 nvarchar(max) null,
zeny nvarchar(max) null,
cash bigint null,
regdate nvarchar(max) null
)

bulk insert #itemmania_ragnarok  from 'C:/Users/Gravity/Desktop/python/raw/itemmania_ragnarok.csv'  with (firstrow=2, fieldterminator = ',' , rowterminator = '\n', codepage='65001' , keepnulls )

insert into mytest.dbo.zeny_selling_monitoring
select * FROM #itemmania_ragnarok 

select * FROM mytest.dbo.zeny_selling_monitoring_2 with (nolock) ; 
'''

itemmania_ragnarok_zero_query = '''

IF OBJECT_ID(N'tempdb..#itemmania_ragnarok_zero') IS NOT NULL
DROP TABLE #itemmania_ragnarok_zero 

CREATE TABLE #itemmania_ragnarok_zero (
game	nvarchar(max) null,
server	nvarchar(max) null,
is_trade_finished nvarchar(max) null,
zeny_sell_title	 nvarchar(max) null,
zeny nvarchar(max) null,
cash bigint null,
regdate nvarchar(max) null
)

bulk insert #itemmania_ragnarok_zero  from 'C:/Users/Gravity/Desktop/python/raw/itemmania_ragnarok_zero.csv'  with (firstrow=2, fieldterminator = ',' , rowterminator = '\n', codepage='65001' , keepnulls )

insert into mytest.dbo.zeny_selling_monitoring
select * FROM #itemmania_ragnarok_zero

select * FROM mytest.dbo.zeny_selling_monitoring_2 with (nolock) ; 
'''

db_processing_query = '''
declare @date date = convert(char(10), getdate(), 120)

insert into [mytest].dbo.[selling_zeny_statistics]
select      convert(char(10), dateadd(dd, -1, @date), 120)
            ,aaaa.server
            ,coalesce(bbbb.trade_cnt, 0) as trade_cnt
            ,coalesce(bbbb.selling_zeny_amount, 0) as selling_zeny_amount
            ,coalesce(bbbb.cash_amount, 0) as cash_amount
            ,coalesce(bbbb.one_billion_zeny, 0) as one_billion_zeny
from        svr_idx  aaaa
left join
(
    select  aaa.regdate
            ,aaa.server
            ,count(1) as 'trade_cnt'
            ,sum(aaa.zeny) as 'selling_zeny_amount'
            ,sum(aaa.cash) as 'cash_amount'
            ,100000000 / (sum(aaa.zeny)/sum(aaa.cash))  as 'one_billion_zeny'
    from 
    (
        select  game
                ,server
                ,is_trade_finished
                ,zeny_sell_title
                ,convert(bigint, billion * 100000000.0 + million) as zeny
                ,cash
                ,convert(char(10), trim(regdate), 120) as regdate
        from 
            (
            select  trim(game) as game
                    ,trim(server) as server
                    ,trim(is_trade_finished) as is_trade_finished
                    ,zeny_sell_title 
                    ,cash
                    ,case
                        when CHARINDEX('억', zeny, 1) > 0
                            then replace(left(zeny, CHARINDEX('억', zeny,1)), '억', '') else 0 end 
                    as billion
                    ,case
                        when CHARINDEX('만', zeny, 1) > 0
                            then  convert(bigint, replace(right(zeny, 5), '만', '') *10000 ) else 0 end  
                    as million
                    ,case 
                        when regdate like N'%전%' then convert(char(10), dateadd(dd, -30, @date), 120) 
                        when regdate like N'%/%'  then convert(char(10), '2021'+'-'+ replace(regdate, '/', '-'), 120) 
                        else regdate
                    end 
                    as regdate
            from    mytest..zeny_selling_monitoring with (nolock)
            ) as aa 
    ) as aaa 
    where
            aaa.regdate = convert(char(10), dateadd(dd, -1, @date), 120)
    group BY
            aaa.regdate,aaa.server 
) as bbbb
on      aaaa.server=bbbb.server

select * from mytest..zeny_selling_monitoring with (nolock) ; 
'''

zeny_quote_query = '''
with    daily_selling_zeny_statistics as ( 
select  regdate
        ,max(case when server ='바포메트' then one_billion_zeny end) as 'baphomet'
        ,max(case when server ='이그드라실' then one_billion_zeny end) as 'Yggdrasil'
        ,max(case when server ='다크로드' then one_billion_zeny end) as 'DarkLord'
        ,max(case when server ='묘르닐' then one_billion_zeny end) as 'Mjolnir'
        ,max(case when server ='브륀힐트' then one_billion_zeny end) as 'Brynhild'
from    [mytest].dbo.[selling_zeny_statistics]
group by
        regdate
) 
,       tb_rownum  as ( 
select  *
        ,ROW_NUMBER() over(order by regdate asc) as rownum
from    daily_selling_zeny_statistics
)
,       tb_rownum_ragnarok_daily_cash as ( 
select  regdate
        ,coalesce((select  baphomet 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.baphomet !=0)
                            ),0) as baphomet
        ,coalesce((select  yggdrasil 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.yggdrasil !=0)
                            ),0) as yggdrasil
        ,coalesce((select  DarkLord 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.DarkLord !=0)
                            ),0) as DarkLord
        ,coalesce((select  Mjolnir 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.Mjolnir !=0)
                            ), 0) as Mjolnir
        ,coalesce((select  Brynhild 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.Brynhild !=0)
                            ),0) as Brynhild    
from    tb_rownum c    
)

SELECT  regdate
        ,format(baphomet,  N'#,0') as  '바포메트'
        ,format(
            coalesce( 
                baphomet - lead(baphomet) over(order by regdate desc)
            , 0),  N'#,0')  as '바포메트_전일 변동량'
        ,format(yggdrasil, N'#,0')  as '이그드라실'
        ,format(
            coalesce(
                yggdrasil - lead(yggdrasil) over(order by regdate desc) 
            , 0), N'#,0')  as '이그드라실_전일 변동량'
        ,format(DarkLord, N'#,0') as '다크로드'
        ,format(
            coalesce(
                DarkLord - lead(DarkLord) over(order by regdate desc) 
            , 0), N'#,0')  as '다크로드_전일 변동량'
        ,format(Mjolnir,  N'#,0') as '묘르닐'
        ,format(
            coalesce(
                Mjolnir - lead(Mjolnir) over(order by regdate desc)
            , 0), N'#,0')  as '묘르닐_전일 변동량'
        ,format(Brynhild,  N'#,0') as '브륀힐트'
        ,format(
            coalesce(
                Brynhild - lead(Brynhild) over(order by regdate desc) 
            , 0), N'#,0') as '브륀힐트_전일 변동량'
from    tb_rownum_ragnarok_daily_cash
'''

zeny_trade_cash_query = '''
select  regdate
        ,format(max(case when server ='바포메트' then trade_cnt end), N'#,0') as '바포메트_거래건수'
		,format(max(case when server ='바포메트' then cash_amount end), N'#,0') as '바포메트_거래금액'
        ,format(max(case when server ='이그드라실' then trade_cnt end), N'#,0') as '이그드라실_거래건수'
        ,format(max(case when server ='이그드라실' then cash_amount end), N'#,0') as '이그드라실_거래금액'
        ,format(max(case when server ='다크로드' then trade_cnt end), N'#,0') as '다크로드_거래건수'
        ,format(max(case when server ='다크로드' then cash_amount end), N'#,0') as '다크로드_거래금액'
        ,format(max(case when server ='묘르닐' then trade_cnt end), N'#,0') as '묘르닐_거래건수'
        ,format(max(case when server ='묘르닐' then cash_amount end), N'#,0') as '묘르닐_거래금액'
		,format(max(case when server ='브륀힐트' then trade_cnt end), N'#,0') as '브륀힐트_거래건수'
		,format(max(case when server ='브륀힐트' then cash_amount end), N'#,0') as '브륀힐트_거래금액'
from    [mytest].dbo.[selling_zeny_statistics]
group by
		regdate
order by
        regdate desc
'''

data1 = pd.read_sql(sql=itembay_ragnarok_query, con=conn)
data2 = pd.read_sql(sql=itembay_ragnarok_zero_query, con=conn)
data3 = pd.read_sql(sql=itemmania_ragnarok_query, con=conn)
data4 = pd.read_sql(sql=itemmania_ragnarok_zero_query, con=conn)
data5 = pd.read_sql(sql=db_processing_query, con=conn)
zeny_quote_summary = pd.read_sql(sql=zeny_quote_query, con=conn)
zeny_trade_cash_summary = pd.read_sql(sql=zeny_trade_cash_query, con=conn)

conn.commit()
conn.close()

now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

zeny_quote_summary.to_csv('C:/Users/Gravity/Desktop/python/raw/statistics/zeny_quote_summary_' + str(nowDate) +'.csv', index=False, encoding='utf-8-sig')
zeny_trade_cash_summary.to_csv('C:/Users/Gravity/Desktop/python/raw/statistics/zeny_trade_cash_summary_'+str(nowDate)+'.csv', index=False, encoding='utf-8-sig')

In [2]:
#주말용 
import  warnings
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys 
from    selenium.webdriver.common.action_chains import ActionChains
from    selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from    bs4      import BeautifulSoup
import  time
import  pyperclip
import  requests
import  datetime
import  pymssql
import  pandas as pd
import  chromedriver_autoinstaller
import  subprocess
from    selenium import webdriver
from    selenium.webdriver.chrome.options import Options
from    selenium.webdriver.common.alert import Alert
import  chromedriver_autoinstaller
import  subprocess
import  shutil 
import  xlrd
import  openpyxl

warnings.filterwarnings('ignore')

conn = pymssql.connect(server='localhost', user='pyuser', password='Dnflskfk12!', database='mytest' , charset='utf8') 

zeny_quote_query = '''
with    daily_selling_zeny_statistics as ( 
select  regdate
        ,max(case when server ='바포메트' then one_billion_zeny end) as 'baphomet'
        ,max(case when server ='이그드라실' then one_billion_zeny end) as 'Yggdrasil'
        ,max(case when server ='다크로드' then one_billion_zeny end) as 'DarkLord'
        ,max(case when server ='묘르닐' then one_billion_zeny end) as 'Mjolnir'
        ,max(case when server ='브륀힐트' then one_billion_zeny end) as 'Brynhild'
from    [mytest].dbo.[selling_zeny_statistics]
group by
        regdate
) 
,       tb_rownum  as ( 
select  *
        ,ROW_NUMBER() over(order by regdate asc) as rownum
from    daily_selling_zeny_statistics
)
,       tb_rownum_ragnarok_daily_cash as ( 
select  regdate
        ,coalesce((select  baphomet 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.baphomet !=0)
                            ),0) as baphomet
        ,coalesce((select  yggdrasil 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.yggdrasil !=0)
                            ),0) as yggdrasil
        ,coalesce((select  DarkLord 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.DarkLord !=0)
                            ),0) as DarkLord
        ,coalesce((select  Mjolnir 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.Mjolnir !=0)
                            ), 0) as Mjolnir
        ,coalesce((select  Brynhild 
            from    tb_rownum a 
                where   a.rownum = 
                    (select  max(b.rownum) 
                        from tb_rownum b 
                            where b.rownum <=c.rownum and b.Brynhild !=0)
                            ),0) as Brynhild    
from    tb_rownum c    
)

SELECT  regdate
        ,format(baphomet,  N'#,0') as  '바포메트'
        ,format(
            coalesce( 
                baphomet - lead(baphomet) over(order by regdate desc)
            , 0),  N'#,0')  as '바포메트_전일 변동량'
        ,format(yggdrasil, N'#,0')  as '이그드라실'
        ,format(
            coalesce(
                yggdrasil - lead(yggdrasil) over(order by regdate desc) 
            , 0), N'#,0')  as '이그드라실_전일 변동량'
        ,format(DarkLord, N'#,0') as '다크로드'
        ,format(
            coalesce(
                DarkLord - lead(DarkLord) over(order by regdate desc) 
            , 0), N'#,0')  as '다크로드_전일 변동량'
        ,format(Mjolnir,  N'#,0') as '묘르닐'
        ,format(
            coalesce(
                Mjolnir - lead(Mjolnir) over(order by regdate desc)
            , 0), N'#,0')  as '묘르닐_전일 변동량'
        ,format(Brynhild,  N'#,0') as '브륀힐트'
        ,format(
            coalesce(
                Brynhild - lead(Brynhild) over(order by regdate desc) 
            , 0), N'#,0') as '브륀힐트_전일 변동량'
from    tb_rownum_ragnarok_daily_cash
'''

zeny_trade_cash_query = '''
select  regdate
        ,format(max(case when server ='바포메트' then trade_cnt end), N'#,0') as '바포메트_거래건수'
		,format(max(case when server ='바포메트' then cash_amount end), N'#,0') as '바포메트_거래금액'
        ,format(max(case when server ='이그드라실' then trade_cnt end), N'#,0') as '이그드라실_거래건수'
        ,format(max(case when server ='이그드라실' then cash_amount end), N'#,0') as '이그드라실_거래금액'
        ,format(max(case when server ='다크로드' then trade_cnt end), N'#,0') as '다크로드_거래건수'
        ,format(max(case when server ='다크로드' then cash_amount end), N'#,0') as '다크로드_거래금액'
        ,format(max(case when server ='묘르닐' then trade_cnt end), N'#,0') as '묘르닐_거래건수'
        ,format(max(case when server ='묘르닐' then cash_amount end), N'#,0') as '묘르닐_거래금액'
		,format(max(case when server ='브륀힐트' then trade_cnt end), N'#,0') as '브륀힐트_거래건수'
		,format(max(case when server ='브륀힐트' then cash_amount end), N'#,0') as '브륀힐트_거래금액'
from    [mytest].dbo.[selling_zeny_statistics]
group by
		regdate
order by
        regdate desc
'''

zeny_quote_summary = pd.read_sql(sql=zeny_quote_query, con=conn)
zeny_trade_cash_summary = pd.read_sql(sql=zeny_trade_cash_query, con=conn)

conn.commit()
conn.close()

now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

zeny_quote_summary.to_csv('C:/Users/Gravity/Desktop/python/raw/statistics/zeny_quote_summary_' + str(nowDate) +'.csv', index=False, encoding='utf-8-sig')
zeny_trade_cash_summary.to_csv('C:/Users/Gravity/Desktop/python/raw/statistics/zeny_trade_cash_summary_'+str(nowDate)+'.csv', index=False, encoding='utf-8-sig')